In [1]:
import pandas as pd
import numpy as np
from scipy import stats

### Задача.

На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [2]:
df = pd.read_excel('AB_Test_ResultsHW8.xlsx')


d:\dcmnts\prg\PyAnalitic\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Изучим данные

In [3]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [5]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


Преобразуем колонки USER_ID и REVENUE для уменьшения занимаемой памяти (здесь не особо нужно, но для общего случая-)

In [12]:
df = df.astype({'USER_ID': np.uint32})
df = df.astype({'REVENUE': np.float32})

Проверим есть ли некорректные данные

In [6]:
df.query('REVENUE < 0').count()

USER_ID         0
VARIANT_NAME    0
REVENUE         0
dtype: int64

Проверим есть ли повторяющиеся пользователи

In [7]:
df.USER_ID.value_counts().sort_values()

USER_ID
4955    1
4867    1
1468    1
3425    1
8098    1
       ..
4879    6
9101    6
668     6
8359    6
5652    6
Name: count, Length: 6324, dtype: int64

In [8]:
df.duplicated(subset=['USER_ID', 'VARIANT_NAME']).sum()

2135

In [12]:
repeated_users = df.groupby('USER_ID')['VARIANT_NAME'].nunique()
users_in_both_groups = repeated_users[repeated_users > 1].index
df_filtered = df[df['USER_ID'].isin(users_in_both_groups)]


In [14]:
df_filtered.sort_values('USER_ID')

,USER_ID,VARIANT_NAME,REVENUE
7409,3,variant,0.00
5377,3,control,0.00
4145,3,variant,0.00
4612,10,variant,0.00
5545,10,control,0.00
...,...,...,...
5915,9996,variant,6.46
2998,9996,control,0.00
6883,10000,variant,0.00
8921,10000,control,0.00


Кажется одной из рекомендаций будет - доработать сплиттер-))

Почти половина данных с пользователями, что попали в обе группы.

При таком количестве избавляться будет не очень хорошо....
Работаем дальше.

In [15]:
df_unique = df[~df['USER_ID'].isin(users_in_both_groups)]

In [16]:
df_unique.describe()

,USER_ID,REVENUE
count,6070.000000,6070.000000
mean,4983.664086,0.107064
std,2897.990118,2.673501
min,2.000000,0.000000
25%,2472.250000,0.000000
50%,4950.000000,0.000000
75%,7501.000000,0.000000
max,9998.000000,196.010000


In [19]:
df.VARIANT_NAME.value_counts()

VARIANT_NAME
variant    5016
control    4984
Name: count, dtype: int64

In [22]:
df_unique.VARIANT_NAME.value_counts()

VARIANT_NAME
variant    3044
control    3026
Name: count, dtype: int64

In [20]:
df.query('VARIANT_NAME == "control"').describe()

,USER_ID,REVENUE
count,4984.000000,4984.000000
mean,4989.436798,0.129013
std,2905.145109,3.007524
min,2.000000,0.000000
25%,2466.000000,0.000000
50%,4964.500000,0.000000
75%,7576.250000,0.000000
max,10000.000000,196.010000


In [21]:
df.query('VARIANT_NAME == "variant"').describe()

,USER_ID,REVENUE
count,5016.000000,5016.000000
mean,4972.776914,0.070070
std,2876.320625,1.314802
min,3.000000,0.000000
25%,2476.500000,0.000000
50%,4958.500000,0.000000
75%,7415.250000,0.000000
max,10000.000000,58.630000


Данные распределены не нормально. Применим не парметрический тест Манна-Уитни.

Нулевая гипотеза - статически значимое различие есть.

In [30]:
gr1 = df[df['VARIANT_NAME'] == 'control']['REVENUE']
gr2 = df[df['VARIANT_NAME'] == 'variant']['REVENUE']

stats.mannwhitneyu(gr1, gr2)



MannwhitneyuResult(statistic=12521564.0, pvalue=0.47825247965294926)

In [32]:
gr1 = df_unique[df_unique['VARIANT_NAME'] == 'control']['REVENUE']
gr2 = df_unique[df_unique['VARIANT_NAME'] == 'variant']['REVENUE']

stats.mannwhitneyu(gr1, gr2)

MannwhitneyuResult(statistic=4622832.0, pvalue=0.2444173738649208)

В обоих случаях (с оригинальными данными и с уникальными) p-value меньше 0.05.
Таким образом делам вывод о том, что статистическая значимая разница между группами есть.
Однако с оригинальными данными значение близко к пороговому.
Сумма дохода в контрольной группе выше.

Рекомендации менеджеру:
1. Оставить всё как есть.
2. Доработать сплиттер ввиду некорректности его работы.
3. Если есть необходимость - переделать тест с новыми сплиттером (и возможно лучше с новыми идеями на доработку сайта).